In [2]:
pip install findspark

Note: you may need to restart the kernel to use updated packages.


In [10]:
# import findspark
# findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Set to the Spark root directory
# from pyspark.sql import SparkSession

# # Initialize Spark session
# spark = SparkSession.builder \
#     .appName("IntrusionDetection") \
#     .getOrCreate()


In [1]:
import findspark
findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Adjust to your Spark root directory

from pyspark.sql import SparkSession

# Initialize Spark session with advanced configurations
spark = SparkSession.builder \
    .appName("IntrusionDetection") \
    .config("spark.master", "spark://10.154.0.2:7077") \
    .config("spark.executor.memory", "3g") \
    .config("spark.driver.memory", "3g") \
    .config("spark.executor.cores", "1") \
    .config("spark.driver.cores", "1") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate() 


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/10 20:03:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Define the path to the dataset in HDFS
hdfs_path = "hdfs://localhost:54310/datasets/NF-UQ-NIDS-v2.csv"  # Adjust the filename if necessary
# Define the path to the dataset in HDFS
#hdfs_path = "hdfs:///datasets/NF-UQ-NIDS-v2/dataset.csv"  # Adjust the filename if necessary


In [3]:
df = spark.read.csv(hdfs_path, header=True, inferSchema=True)

In [12]:
# Display the schema of the DataFrame
df.printSchema()

# Show summary statistics of the dataset
df.describe().show()


root
 |-- IPV4_SRC_ADDR: string (nullable = true)
 |-- L4_SRC_PORT: integer (nullable = true)
 |-- IPV4_DST_ADDR: string (nullable = true)
 |-- L4_DST_PORT: integer (nullable = true)
 |-- PROTOCOL: integer (nullable = true)
 |-- L7_PROTO: double (nullable = true)
 |-- IN_BYTES: integer (nullable = true)
 |-- IN_PKTS: integer (nullable = true)
 |-- OUT_BYTES: integer (nullable = true)
 |-- OUT_PKTS: integer (nullable = true)
 |-- TCP_FLAGS: integer (nullable = true)
 |-- CLIENT_TCP_FLAGS: integer (nullable = true)
 |-- SERVER_TCP_FLAGS: integer (nullable = true)
 |-- FLOW_DURATION_MILLISECONDS: integer (nullable = true)
 |-- DURATION_IN: integer (nullable = true)
 |-- DURATION_OUT: integer (nullable = true)
 |-- MIN_TTL: integer (nullable = true)
 |-- MAX_TTL: integer (nullable = true)
 |-- LONGEST_FLOW_PKT: integer (nullable = true)
 |-- SHORTEST_FLOW_PKT: integer (nullable = true)
 |-- MIN_IP_PKT_LEN: integer (nullable = true)
 |-- MAX_IP_PKT_LEN: integer (nullable = true)
 |-- SRC_TO

24/11/07 19:58:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-------------+------------------+-------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+------------------+--------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+--------------------+------------------+--------+---------------+
|summary|IPV4_SRC_ADDR|       L4_SRC_PORT|IPV4_DST_ADDR|     

In [11]:
# Import necessary functions from PySpark
from pyspark.sql.functions import col, sum as spark_sum, countDistinct

# Check for missing values in each column
missing_values = df.select([(spark_sum(col(c).isNull().cast("int")).alias(c)) for c in df.columns])
missing_values.show()

# Get the unique count of values for each column
unique_counts = df.agg(*[countDistinct(col(c)).alias(c) for c in df.columns])
unique_counts.show()


24/11/08 14:42:05 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------+-----------+-------------+-----------+--------+--------+--------+-------+---------+--------+---------+----------------+----------------+--------------------------+-----------+------------+-------+-------+----------------+-----------------+--------------+--------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+--------------+---------------+---------+--------------+------------+--------------+--------------+--------------------+-----+------+-------+
|IPV4_SRC_ADDR|L4_SRC_PORT|IPV4_DST_ADDR|L4_DST_PORT|PROTOCOL|L7_PROTO|IN_BYTES|IN_PKTS|OUT_BYTES|OUT_PKTS|TCP_FLAGS|CLIENT_TCP_FLAGS|SERVER_TCP_FLAGS|FLOW_DURATION_MILLISECONDS|DURATION_IN|DURATION_OUT|MIN_TTL|MAX_TTL|LONGEST_FLOW_PKT|SHORTEST_FLOW_

+-------------+-----------+-------------+-----------+--------+--------+--------+-------+---------+--------+---------+----------------+----------------+--------------------------+-----------+------------+-------+-------+----------------+-----------------+--------------+--------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+--------------+---------------+---------+--------------+------------+--------------+--------------+--------------------+-----+------+-------+
|IPV4_SRC_ADDR|L4_SRC_PORT|IPV4_DST_ADDR|L4_DST_PORT|PROTOCOL|L7_PROTO|IN_BYTES|IN_PKTS|OUT_BYTES|OUT_PKTS|TCP_FLAGS|CLIENT_TCP_FLAGS|SERVER_TCP_FLAGS|FLOW_DURATION_MILLISECONDS|DURATION_IN|DURATION_OUT|MIN_TTL|MAX_TTL|LONGEST_FLOW_PKT|SHORTEST_FLOW_

In [5]:
from pyspark.sql.functions import col

# Define categorical and continuous features based on unique value counts and feature characteristics
categorical_features = [
    "PROTOCOL", "TCP_FLAGS", "CLIENT_TCP_FLAGS", "SERVER_TCP_FLAGS", 
    "ICMP_TYPE", "ICMP_IPV4_TYPE", "DNS_QUERY_ID", "DNS_QUERY_TYPE", 
    "Label", "Attack", "Dataset"
]

continuous_features = [
    "L4_SRC_PORT", "L4_DST_PORT", "IN_BYTES", "IN_PKTS", "OUT_BYTES", 
    "OUT_PKTS", "FLOW_DURATION_MILLISECONDS", "DURATION_IN", "DURATION_OUT", 
    "MIN_TTL", "MAX_TTL", "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", 
    "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "SRC_TO_DST_SECOND_BYTES", 
    "DST_TO_SRC_SECOND_BYTES", "RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS", 
    "RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS", "SRC_TO_DST_AVG_THROUGHPUT", 
    "DST_TO_SRC_AVG_THROUGHPUT", "NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", 
    "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES", 
    "TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT", "FTP_COMMAND_RET_CODE"
]

# Show summary statistics for continuous features
df.select([col(c) for c in continuous_features]).describe().show()

# Show unique value counts for categorical features
for feature in categorical_features:
    df.select(feature).distinct().show(truncate=False)


24/11/09 09:34:41 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+------------------+-----------------+------------------+-----------------+--------------------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+-----------------------+-----------------------+----------------------+---------------------+-----------------------+----------------------+-------------------------+-------------------------+------------------------+-------------------------+-------------------------+--------------------------+---------------------------+------------------+------------------+--------------------+
|summary|       L4_SRC_PORT|      L4_DST_PORT|          IN_BYTES|          IN_PKTS|         OUT_BYTES|         OUT_PKTS|FLOW_DURATION_MILLISECONDS|      DURATION_IN|      DURATION_OUT|           MIN_TTL|          MAX_TTL|  LONGEST_FLOW_PKT|SHORTEST_FLOW_PKT|    MIN_IP_PKT_LEN|    MAX_IP_PKT_LEN|SRC_TO_DST_SECOND_BYTES|

+--------+
|PROTOCOL|
+--------+
|244     |
|65      |
|203     |
|130     |
|161     |
|248     |
|13      |
|232     |
|67      |
|171     |
|25      |
|190     |
|198     |
|126     |
|18      |
|110     |
|150     |
|225     |
|238     |
|157     |
+--------+
only showing top 20 rows



+---------+
|TCP_FLAGS|
+---------+
|18       |
|25       |
|218      |
|223      |
|19       |
|222      |
|23       |
|30       |
|17       |
|61       |
|16       |
|194      |
|20       |
|6        |
|29       |
|82       |
|43       |
|152      |
|4        |
|2        |
+---------+
only showing top 20 rows



+----------------+
|CLIENT_TCP_FLAGS|
+----------------+
|18              |
|25              |
|198             |
|218             |
|222             |
|19              |
|6               |
|30              |
|16              |
|223             |
|17              |
|194             |
|20              |
|23              |
|43              |
|29              |
|152             |
|2               |
|31              |
|4               |
+----------------+
only showing top 20 rows



...

+------------+
|DNS_QUERY_ID|
+------------+
|25376       |
|51839       |
|52127       |
|54862       |
|62215       |
|248         |
|45824       |
|27344       |
|5634        |
|10170       |
|15044       |
|18934       |
|26853       |
|39485       |
|21360       |
|58462       |
|53972       |
|21858       |
|5942        |
|2224        |
+------------+
only showing top 20 rows



+--------------+
|DNS_QUERY_TYPE|
+--------------+
|12            |
|46            |
|255           |
|43            |
|16            |
|48            |
|28            |
|2             |
|0             |
|1             |
|33            |
|52            |
|252           |
|15            |
|8465          |
|251           |
|32769         |
|6             |
|4096          |
|55937         |
+--------------+
only showing top 20 rows



+-----+
|Label|
+-----+
|1    |
|0    |
+-----+



+--------------+
|Attack        |
+--------------+
|Benign        |
|DDoS          |
|Fuzzers       |
|ransomware    |
|Reconnaissance|
|injection     |
|Exploits      |
|DoS           |
|Bot           |
|Infilteration |
|Generic       |
|Analysis      |
|mitm          |
|Worms         |
|scanning      |
|xss           |
|password      |
|Theft         |
|Brute Force   |
|Backdoor      |
+--------------+
only showing top 20 rows



+---------------------+
|Dataset              |
+---------------------+
|NF-BoT-IoT-v2        |
|NF-ToN-IoT-v2        |
|NF-CSE-CIC-IDS2018-v2|
|NF-UNSW-NB15-v2      |
+---------------------+



In [7]:
from pyspark.sql.functions import col, when

# Define outlier thresholds based on domain knowledge or visual inspection
outlier_thresholds = {
    "IN_BYTES": 1e6,
    "OUT_BYTES": 1e6,
    "SRC_TO_DST_SECOND_BYTES": 1e6,
    "DST_TO_SRC_SECOND_BYTES": 1e6,
    "SRC_TO_DST_AVG_THROUGHPUT": 1e8,
    "DST_TO_SRC_AVG_THROUGHPUT": 1e8
}

# Cap outliers in specified columns
for col_name, threshold in outlier_thresholds.items():
    df = df.withColumn(
        col_name, 
        when(col(col_name) > threshold, threshold).otherwise(col(col_name))
    )

# Verify the changes with describe
df.select(list(outlier_thresholds.keys())).describe().show()


+-------+------------------+------------------+-----------------------+-----------------------+-------------------------+-------------------------+
|summary|          IN_BYTES|         OUT_BYTES|SRC_TO_DST_SECOND_BYTES|DST_TO_SRC_SECOND_BYTES|SRC_TO_DST_AVG_THROUGHPUT|DST_TO_SRC_AVG_THROUGHPUT|
+-------+------------------+------------------+-----------------------+-----------------------+-------------------------+-------------------------+
|  count|          75987976|          75987976|               75987976|               75987976|                 75987976|                 75987976|
|   mean| 684.9687731516891|1931.1599471605875|     53844.412021870405|      8083.790723982436|        2551493.682922677|        3930067.335627521|
| stddev|10906.559260381766|28458.865621628727|     157701.53738160757|      79557.19427151275|        7246163.087712197|     1.3677411811359156E7|
|    min|               1.0|               0.0|                    0.0|                    0.0|                 

In [8]:
!pip install numpy

     |████████████████████████████████| 17.3 MB 12.6 MB/s eta 0:00:01


In [8]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Define StringIndexers for categorical features
indexers = [
    StringIndexer(inputCol="PROTOCOL", outputCol="PROTOCOL_IDX"),
    StringIndexer(inputCol="TCP_FLAGS", outputCol="TCP_FLAGS_IDX"),
    StringIndexer(inputCol="CLIENT_TCP_FLAGS", outputCol="CLIENT_TCP_FLAGS_IDX"),
    StringIndexer(inputCol="SERVER_TCP_FLAGS", outputCol="SERVER_TCP_FLAGS_IDX"),
    StringIndexer(inputCol="Attack", outputCol="Attack_IDX"),
    StringIndexer(inputCol="Dataset", outputCol="Dataset_IDX")
]

# Define OneHotEncoders for indexed features
encoders = [
    OneHotEncoder(inputCol="PROTOCOL_IDX", outputCol="PROTOCOL_VEC"),
    OneHotEncoder(inputCol="TCP_FLAGS_IDX", outputCol="TCP_FLAGS_VEC"),
    OneHotEncoder(inputCol="CLIENT_TCP_FLAGS_IDX", outputCol="CLIENT_TCP_FLAGS_VEC"),
    OneHotEncoder(inputCol="SERVER_TCP_FLAGS_IDX", outputCol="SERVER_TCP_FLAGS_VEC"),
    OneHotEncoder(inputCol="Attack_IDX", outputCol="Attack_VEC"),
    OneHotEncoder(inputCol="Dataset_IDX", outputCol="Dataset_VEC")
]

# Combine stages into a pipeline
pipeline = Pipeline(stages=indexers + encoders)

# Fit and transform the data
df_encoded = pipeline.fit(df).transform(df)

# Display the resulting columns to confirm the encoding
df_encoded.select("PROTOCOL_VEC", "TCP_FLAGS_VEC", "Attack_VEC", "Dataset_VEC").show(5, truncate=False)


+---------------+--------------+--------------+-------------+
|PROTOCOL_VEC   |TCP_FLAGS_VEC |Attack_VEC    |Dataset_VEC  |
+---------------+--------------+--------------+-------------+
|(255,[0],[1.0])|(52,[1],[1.0])|(20,[2],[1.0])|(3,[0],[1.0])|
|(255,[0],[1.0])|(52,[2],[1.0])|(20,[2],[1.0])|(3,[0],[1.0])|
|(255,[0],[1.0])|(52,[2],[1.0])|(20,[0],[1.0])|(3,[2],[1.0])|
|(255,[0],[1.0])|(52,[2],[1.0])|(20,[0],[1.0])|(3,[2],[1.0])|
|(255,[0],[1.0])|(52,[2],[1.0])|(20,[0],[1.0])|(3,[2],[1.0])|
+---------------+--------------+--------------+-------------+
only showing top 5 rows



In [9]:
from pyspark.ml.feature import VectorAssembler

# Define the list of continuous and encoded feature columns
continuous_features = [
    "L4_SRC_PORT", "L4_DST_PORT", "IN_BYTES", "IN_PKTS", "OUT_BYTES", 
    "OUT_PKTS", "FLOW_DURATION_MILLISECONDS", "DURATION_IN", "DURATION_OUT", 
    "MIN_TTL", "MAX_TTL", "LONGEST_FLOW_PKT", "SHORTEST_FLOW_PKT", 
    "MIN_IP_PKT_LEN", "MAX_IP_PKT_LEN", "SRC_TO_DST_SECOND_BYTES", 
    "DST_TO_SRC_SECOND_BYTES", "RETRANSMITTED_IN_BYTES", "RETRANSMITTED_IN_PKTS", 
    "RETRANSMITTED_OUT_BYTES", "RETRANSMITTED_OUT_PKTS", "SRC_TO_DST_AVG_THROUGHPUT", 
    "DST_TO_SRC_AVG_THROUGHPUT", "NUM_PKTS_UP_TO_128_BYTES", "NUM_PKTS_128_TO_256_BYTES", 
    "NUM_PKTS_256_TO_512_BYTES", "NUM_PKTS_512_TO_1024_BYTES", "NUM_PKTS_1024_TO_1514_BYTES", 
    "TCP_WIN_MAX_IN", "TCP_WIN_MAX_OUT", "FTP_COMMAND_RET_CODE"
]
encoded_features = ["PROTOCOL_VEC", "TCP_FLAGS_VEC", "CLIENT_TCP_FLAGS_VEC", 
                    "SERVER_TCP_FLAGS_VEC", "Attack_VEC", "Dataset_VEC"]

# Assemble all features into a single vector column
assembler = VectorAssembler(
    inputCols=continuous_features + encoded_features,
    outputCol="features"
)

# Transform the data to add the 'features' column
df_final = assembler.transform(df_encoded)

# Select the features and label columns for the final dataset
df_final = df_final.select("features", "Label")

# Show a sample of the final dataset
df_final.show(5, truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                     |Label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|(454,[0,1,2,3,6,7,9,10,11,12,14,15,17,18,21,24,28,31,287,339,388,433,451],[65389.0,80.0,420.0,3.0,4293092.0,1875.0,64.0,64.0,140.0,140.0,140.0,140280.0,140.0,1.0,1120000.0,3.0,512.0,1.0,1.0,1.0,1.0,1.0,1.0])              |1    |
|(454,[0,1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,22,23,24,28,31,288,339,390,433,451

In [10]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

# Assuming 'df_final' is a DataFrame already defined and available for splitting
train_df, test_df = df_final.randomSplit([0.7, 0.3], seed=42)

# Save the DataFrames to HDFS under the datasets directory
train_df.write.format("parquet").mode("overwrite").save("hdfs://localhost:54310/datasets/train_df")
test_df.write.format("parquet").mode("overwrite").save("hdfs://localhost:54310/datasets/test_df")

# Display the number of records in each dataset
print("Training Dataset Count: " + str(train_df.count()))
print("Test Dataset Count: " + str(test_df.count()))


Training Dataset Count: 53186564


Test Dataset Count: 22801412


In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler

import findspark
findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Adjust to your specific Spark installation path

from pyspark.sql import SparkSession

# Initialize Spark session with adjusted configurations
spark = SparkSession.builder \
    .appName("RandomForestModel") \
    .config("spark.executor.memory", "3g")\
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.memoryOverhead", "512m")\
    .config("spark.executor.cores", "1")\
    .config("spark.executor.instances", "3")\
    .config("spark.driver.maxResultSize", "1g")\
    .config("spark.default.parallelism", "12")\
    .config("spark.sql.shuffle.partitions", "12")\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.5")\
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


# Load the training and testing data from HDFS
train_df = spark.read.format("parquet").load("hdfs://localhost:54310/datasets/train_df")
test_df = spark.read.format("parquet").load("hdfs://localhost:54310/datasets/test_df")

# Initialize the RandomForest model
rf = RandomForestClassifier(featuresCol='features', labelCol='Label', numTrees=10)

# Train the model
rf_model = rf.fit(train_df)

# Predict on the test data
predictions = rf_model.transform(test_df)

# Show some prediction results
predictions.select("features", "Label", "prediction").show(5)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 12:20:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


+--------------------+-----+----------+
|            features|Label|prediction|
+--------------------+-----+----------+
|(454,[0,1,2,3,4,5...|    0|       0.0|
|(454,[0,1,2,3,4,5...|    0|       0.0|
|(454,[0,1,2,3,4,5...|    0|       0.0|
|(454,[0,1,2,3,4,5...|    0|       0.0|
|(454,[0,1,2,3,4,5...|    0|       0.0|
+--------------------+-----+----------+
only showing top 5 rows



In [2]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Evaluate the model using BinaryClassificationEvaluator for ROC AUC
binary_evaluator = BinaryClassificationEvaluator(labelCol="Label")
auc = binary_evaluator.evaluate(predictions, {binary_evaluator.metricName: "areaUnderROC"})
print("Area under ROC = %g" % auc)

# Evaluate the model using MulticlassClassificationEvaluator for Accuracy and F1-Score
multi_evaluator = MulticlassClassificationEvaluator(labelCol="Label", metricName="accuracy")
accuracy = multi_evaluator.evaluate(predictions)
print("Accuracy = %g" % accuracy)

f1 = multi_evaluator.setMetricName("f1").evaluate(predictions)
print("F1 Score = %g" % f1)


Area under ROC = 0.997644


Accuracy = 0.978895


F1 Score = 0.978871


In [ ]:
# # Load the training and testing data from HDFS
# train_df = spark.read.format("parquet").load("hdfs://localhost:54310/datasets/train_df")
# test_df = spark.read.format("parquet").load("hdfs://localhost:54310/datasets/test_df")


In [3]:
from pyspark.mllib.evaluation import MulticlassMetrics

# Convert DataFrame predictions to RDD to use with MulticlassMetrics
predictionAndLabels = predictions.select("prediction", "Label").rdd.map(lambda x: (float(x[0]), float(x[1])))

# Compute confusion matrix
metrics = MulticlassMetrics(predictionAndLabels)
confusion_matrix = metrics.confusionMatrix().toArray()

print("Confusion Matrix:")
print(confusion_matrix)


/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion Matrix:
[[ 7274911.   274516.]
 [  206702. 15045283.]]


In [12]:
# Specify the path in HDFS where you want to save the model
model_path = "hdfs://localhost:54310/datasets/randomforest_model"

# Save the trained RandomForest model
rf_model.save(model_path)


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming 'confusion_matrix' is your numpy array from earlier
# For example:
# confusion_matrix = np.array([[7274911, 274516], [206702, 15045283]])

fig, ax = plt.subplots(figsize=(8, 8))  # Set figure size for better visibility
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues", ax=ax,
            xticklabels=["Predicted Negative", "Predicted Positive"],
            yticklabels=["Actual Negative", "Actual Positive"])

ax.set_title('Confusion Matrix')  # Title for the heatmap
ax.set_xlabel('Predicted Labels')  # Label for x-axis
ax.set_ylabel('Actual Labels')  # Label for y-axis

plt.show()


ModuleNotFoundError: No module named 'seaborn'

In [4]:
# Calculating additional metrics from the confusion matrix
total = confusion_matrix.sum()
accuracy = (confusion_matrix[0, 0] + confusion_matrix[1, 1]) / total
precision = confusion_matrix[1, 1] / (confusion_matrix[1, 1] + confusion_matrix[0, 1])
recall = confusion_matrix[1, 1] / (confusion_matrix[1, 1] + confusion_matrix[1, 0])
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")


Accuracy: 0.9789
Precision: 0.9821
Recall: 0.9864
F1 Score: 0.9843


In [1]:
import findspark
findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Adjust to your specific Spark installation path

from pyspark.sql import SparkSession

# Initialize Spark session with adjusted configurations
spark = SparkSession.builder \
    .appName("AnomalyDetection") \
    .config("spark.executor.memory", "3g")\
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.memoryOverhead", "512m")\
    .config("spark.executor.cores", "1")\
    .config("spark.executor.instances", "3")\
    .config("spark.driver.maxResultSize", "1g")\
    .config("spark.default.parallelism", "12")\
    .config("spark.sql.shuffle.partitions", "12")\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.5")\
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


hdfs_path = "hdfs://localhost:54310/datasets/NF-UQ-NIDS-v2.csv" 
df = spark.read.csv(hdfs_path, header=True, inferSchema=True)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 18:41:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Print schema to verify available columns
df.printSchema()


root
 |-- IPV4_SRC_ADDR: string (nullable = true)
 |-- L4_SRC_PORT: integer (nullable = true)
 |-- IPV4_DST_ADDR: string (nullable = true)
 |-- L4_DST_PORT: integer (nullable = true)
 |-- PROTOCOL: integer (nullable = true)
 |-- L7_PROTO: double (nullable = true)
 |-- IN_BYTES: integer (nullable = true)
 |-- IN_PKTS: integer (nullable = true)
 |-- OUT_BYTES: integer (nullable = true)
 |-- OUT_PKTS: integer (nullable = true)
 |-- TCP_FLAGS: integer (nullable = true)
 |-- CLIENT_TCP_FLAGS: integer (nullable = true)
 |-- SERVER_TCP_FLAGS: integer (nullable = true)
 |-- FLOW_DURATION_MILLISECONDS: integer (nullable = true)
 |-- DURATION_IN: integer (nullable = true)
 |-- DURATION_OUT: integer (nullable = true)
 |-- MIN_TTL: integer (nullable = true)
 |-- MAX_TTL: integer (nullable = true)
 |-- LONGEST_FLOW_PKT: integer (nullable = true)
 |-- SHORTEST_FLOW_PKT: integer (nullable = true)
 |-- MIN_IP_PKT_LEN: integer (nullable = true)
 |-- MAX_IP_PKT_LEN: integer (nullable = true)
 |-- SRC_TO

In [5]:
# Select the required columns for temporal analysis and anomaly detection
df_anomaly = df.select(
    "IPV4_SRC_ADDR", 
    "IPV4_DST_ADDR", 
    "FLOW_DURATION_MILLISECONDS", 
    "IN_BYTES", 
    "OUT_BYTES", 
    "IN_PKTS", 
    "OUT_PKTS", 
    "Label"
)

# Verify the contents of df_anomaly
df_anomaly.show(5)


+---------------+-------------+--------------------------+--------+---------+-------+--------+-----+
|  IPV4_SRC_ADDR|IPV4_DST_ADDR|FLOW_DURATION_MILLISECONDS|IN_BYTES|OUT_BYTES|IN_PKTS|OUT_PKTS|Label|
+---------------+-------------+--------------------------+--------+---------+-------+--------+-----+
|192.168.100.148|192.168.100.7|                   4293092|     420|        0|      3|       0|    1|
|192.168.100.148|192.168.100.5|                   4294499|     280|       40|      2|       1|    1|
|   192.168.1.31| 192.168.1.79|                         0|      44|       40|      1|       1|    0|
|   192.168.1.34| 192.168.1.79|                         0|      44|       40|      1|       1|    0|
|   192.168.1.30|192.168.1.152|                         0|      44|       40|      1|       1|    0|
+---------------+-------------+--------------------------+--------+---------+-------+--------+-----+
only showing top 5 rows



In [6]:
# Save the optimized DataFrame in HDFS for future use
df_anomaly.write.parquet("hdfs://localhost:54310/datasets/anomaly_detection_df")


In [1]:
import findspark
findspark.init('/opt/spark/spark-3.4.4-bin-hadoop3')  # Adjust to your specific Spark installation path

from pyspark.sql import SparkSession

# Initialize Spark session with adjusted configurations
spark = SparkSession.builder \
    .appName("AnomalyDetection") \
    .config("spark.executor.memory", "3g")\
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.memoryOverhead", "512m")\
    .config("spark.executor.cores", "1")\
    .config("spark.executor.instances", "3")\
    .config("spark.driver.maxResultSize", "1g")\
    .config("spark.default.parallelism", "12")\
    .config("spark.sql.shuffle.partitions", "12")\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.5")\
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")\
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")


df_anomaly = spark.read.parquet("hdfs://localhost:54310/datasets/anomaly_detection_df")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/11 18:56:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

# Define a window specification partitioned by source-destination, without ordering by a specific column
window_spec = Window.partitionBy("IPV4_SRC_ADDR", "IPV4_DST_ADDR").orderBy("IPV4_SRC_ADDR", "IPV4_DST_ADDR")

# Generate synthetic timestamps by incrementing within each partition
df_anomaly = df_anomaly.withColumn("synthetic_timestamp", row_number().over(window_spec) - 1)

# Display the results to check if timestamps are generated as expected
df_anomaly.select("IPV4_SRC_ADDR", "IPV4_DST_ADDR", "FLOW_DURATION_MILLISECONDS", "synthetic_timestamp").show(5)


+-------------+-------------+--------------------------+-------------------+
|IPV4_SRC_ADDR|IPV4_DST_ADDR|FLOW_DURATION_MILLISECONDS|synthetic_timestamp|
+-------------+-------------+--------------------------+-------------------+
|0.109.229.230|192.168.1.152|                         0|                  0|
|    0.14.2.72|192.168.1.194|                         0|                  0|
|0.163.152.173|192.168.1.190|                         0|                  0|
|   1.0.126.42| 172.31.67.29|                         0|                  0|
|   1.0.176.98| 172.31.67.96|                         0|                  0|
+-------------+-------------+--------------------------+-------------------+
only showing top 5 rows



In [12]:
# Collect the relevant columns into a Pandas DataFrame for processing
df_anomaly_pd = df_anomaly.select("IPV4_SRC_ADDR", "IPV4_DST_ADDR", "FLOW_DURATION_MILLISECONDS").toPandas()


24/11/11 19:17:13 ERROR Executor: Exception in task 6.0 in stage 35.0 (TID 162)]
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:61)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:348)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$.$anonfun$serializeToChunkedBuffer$1$adapted(SerializerHelper.scala:40)
	at org.apache.spark.serializer.SerializerHelper$$$Lambda$1682/0x0000000840cfd040.apply(Unknown Source)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.allocateNewChunkIfNeeded(ChunkedByteBufferOutputStream.scala:87)
	at org.apache.spark.util.io.ChunkedByteBufferOutputStream.write(ChunkedByteBufferOutputStream.scala:75)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1849)
	at java.base/java.io.ObjectOutputStream.write(ObjectOutputStream.java:708)


ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
# Check for any non-zero values in FLOW_DURATION_MILLISECONDS
df_anomaly.select("FLOW_DURATION_MILLISECONDS").distinct().show(5)

# Alternatively, count the rows where FLOW_DURATION_MILLISECONDS is zero and non-zero
zero_count = df_anomaly.filter(df_anomaly["FLOW_DURATION_MILLISECONDS"] == 0).count()
non_zero_count = df_anomaly.filter(df_anomaly["FLOW_DURATION_MILLISECONDS"] > 0).count()

print(f"Zero duration count: {zero_count}")
print(f"Non-zero duration count: {non_zero_count}")


+--------------------------+
|FLOW_DURATION_MILLISECONDS|
+--------------------------+
|                   4293092|
|                   4294655|
|                   4293436|
|                   4293451|
|                   4293572|
+--------------------------+
only showing top 5 rows



Zero duration count: 35017386
Non-zero duration count: 40970590
